In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [5]:
lista_ligas = ['https://fbref.com/en/comps/12/2021-2022/2021-2022-La-Liga-Stats',
               "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats",
               "https://fbref.com/en/comps/11/2021-2022/2021-2022-Serie-A-Stats",
               "https://fbref.com/en/comps/13/2021-2022/2021-2022-Ligue-1-Stats",
               "https://fbref.com/en/comps/20/2021-2022/2021-2022-Bundesliga-Stats",
               ]

years = list(range(2021, 2015, -1))
all_matches = []
log_error = []
log = []
for year in years:
    for indice,liga in enumerate(lista_ligas):
        data = requests.get(liga)
        soup = BeautifulSoup(data.text)
        
   
        try:
            standings_table = soup.select('table.stats_table')[0]
        except Exception as e:
            log_error.append(f'error:{e},{lista_ligas[indice]},{team_url}')
            continue
        

        links = [l.get("href") for l in standings_table.find_all('a')]
        links = [l for l in links if '/squads/' in l]
        team_urls = [f"https://fbref.com{l}" for l in links]


        previous_season = soup.select("a.prev")[0].get("href")
        print(lista_ligas[indice])
        
        time.sleep(1)
        for team_url in team_urls:
            print(team_url)
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            data = requests.get(team_url)
            time.sleep(1)
            try:
                matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
                matches = matches[['Date', 'Time', 'Comp', 'Round', 'Venue', 'Result', 'GF', 'GA',
       'Opponent',  'Poss', 'Formation',
        ]]
                soup = BeautifulSoup(data.text)
                links = [l.get("href") for l in soup.find_all('a')]
                time.sleep(1)
            except Exception as e:
                log_error.append(f'error:{e},{lista_ligas[indice]},{team_url}')
                continue

            try:
                links_shooting = [l for l in links if l and 'all_comps/shooting/' in l]
                data_shooting = requests.get(f"https://fbref.com{links_shooting[0]}")
                shooting = pd.read_html(data_shooting.text, match="Shooting")[0]
                shooting.columns = shooting.columns.droplevel()
                time.sleep(1)
            except Exception as e:
                log_error.append(f'error:{e},{lista_ligas[indice]},{team_url}')
                continue

     

            try:
                
                links_defensive = [l for l in links if l and 'all_comps/defense/' in l]
                if len(links_defensive)>0:
                  data_defensive = requests.get(f"https://fbref.com{links_defensive[0]}")
                  defensive = pd.read_html(data_defensive.text, match="Defensive Actions")[0]
                  defensive.columns = defensive.columns.droplevel()
                  time.sleep(1)
                else:
                    try:
                      links_misc = [l for l in links if l and 'all_comps/misc/' in l]
                      data_misc = requests.get(f"https://fbref.com{links_misc[0]}")
                      defensive = pd.read_html(data_misc.text, match="Miscellaneous Stats")[0]
                      defensive.columns = defensive.columns.droplevel()
                      time.sleep(1)
                    except Exception as e:
          
                      continue

            except Exception as e:
               
                continue
            
            





            try:
                team_data = matches.merge(defensive[["Date",	'TklW',	'Int'	 ]], on="Date")
            except ValueError:
                continue

            try:
                team_data = team_data.merge(shooting[["Date", "Sh", "SoT", "Dist",  "PK", "PKatt",'xG']], on="Date")
            except Exception as e:
                team_data = team_data.merge(shooting[["Date", "Sh", "SoT", "Dist",  "PK", "PKatt"]], on="Date")
                continue
         


            team_data["Season"] = year
            team_data["Team"] = team_name
            all_matches.append(team_data)
            time.sleep(2)
            pass

    
     
        lista_ligas[indice] = f"https://fbref.com{previous_season}"
        print('cambie a '+lista_ligas[indice])

https://fbref.com/en/comps/12/2021-2022/2021-2022-La-Liga-Stats
https://fbref.com/en/squads/53a2f082/2021-2022/Real-Madrid-Stats
https://fbref.com/en/squads/206d90db/2021-2022/Barcelona-Stats
https://fbref.com/en/squads/db3b9613/2021-2022/Atletico-Madrid-Stats
https://fbref.com/en/squads/ad2be733/2021-2022/Sevilla-Stats
https://fbref.com/en/squads/fc536746/2021-2022/Real-Betis-Stats
https://fbref.com/en/squads/e31d1cd9/2021-2022/Real-Sociedad-Stats
https://fbref.com/en/squads/2a8183b3/2021-2022/Villarreal-Stats
https://fbref.com/en/squads/2b390eca/2021-2022/Athletic-Club-Stats
https://fbref.com/en/squads/dcc91a7b/2021-2022/Valencia-Stats
https://fbref.com/en/squads/03c57e2b/2021-2022/Osasuna-Stats
https://fbref.com/en/squads/f25da7fb/2021-2022/Celta-Vigo-Stats
https://fbref.com/en/squads/98e8af82/2021-2022/Rayo-Vallecano-Stats
https://fbref.com/en/squads/6c8b07df/2021-2022/Elche-Stats
https://fbref.com/en/squads/a8661628/2021-2022/Espanyol-Stats
https://fbref.com/en/squads/7848bd64/202

In [1]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df = match_df[['date', 'time', 'season', 'team', 'comp', 'round', 'venue', 'result', 'gf', 'ga',
       'opponent', 'poss', 'formation', 'tklw', 'int', 'sh', 'sot', 'dist',
       'pk', 'pkatt', 'xg']]
match_df.to_csv('matches.csv')

NameError: ignored